In [1]:
import pandas as pd
pd.set_option('display.max_columns', None) 
import numpy as np
import pickle

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
input_path = 'H:\RediMinds\VCQI'
df_filter2_codes = pd.read_csv(input_path+"\VCQI_POST_OP_clean.csv")

In [3]:
# sepearte target variable from predictors for furthur processing
X = df_filter2_codes.drop(columns=['POST_OP_COMPLICATIONS'])
Y = df_filter2_codes['POST_OP_COMPLICATIONS']
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3312 entries, 0 to 3311
Data columns (total 33 columns):
PROC NAME                          3312 non-null int64
GENDER                             3312 non-null int64
AGE AT SURGERY                     3312 non-null float64
MARITAL STATUS                     3312 non-null int64
RACE                               3312 non-null int64
WEIGHT                             3312 non-null float64
HEIGHT                             3312 non-null float64
BMI                                3312 non-null float64
CLINICAL SIZE (mm)                 3312 non-null float64
SYMPTOMS                           3312 non-null int64
SOLITARY KIDNEY                    3312 non-null int64
BILATERALITY OF TUMOR              3312 non-null int64
SIDE OF TUMOR                      3312 non-null int64
SIDE OF SURGERY                    3312 non-null int64
FACE                               3312 non-null int64
TUMOR lOCATION                     3312 non-null int64
PRE

In [5]:
# Create dummy variables
with open (input_path+'\POST_OP_cat_col', 'rb') as fp:
    cat_col = pickle.load(fp)


nn_data = X.copy()
for col in cat_col:
    if not col == 'POST_OP_COMLICATIONS':
        nn_data = nn_data.merge(pd.get_dummies(X[col], prefix=col), how = 'left', left_index = True, right_index =True) # create dummy variables
        nn_data = nn_data.drop(labels = col, axis = 'columns') # drop original variables

In [6]:
nn_data.head()

,AGE AT SURGERY,WEIGHT,HEIGHT,BMI,CLINICAL SIZE (mm),PRE-OP HB,PRE-OP HT,PRE-OP WBC,PRE-OP CREAT,PRE-OP EGFR,NO OF LESIONS,PROC NAME_0,GENDER_0,GENDER_1,GENDER_2,MARITAL STATUS_0,MARITAL STATUS_1,MARITAL STATUS_2,MARITAL STATUS_3,MARITAL STATUS_4,RACE_0,RACE_1,RACE_2,RACE_3,RACE_4,SYMPTOMS_0,SYMPTOMS_1,SYMPTOMS_2,SYMPTOMS_3,SOLITARY KIDNEY_0,SOLITARY KIDNEY_1,SOLITARY KIDNEY_2,BILATERALITY OF TUMOR_0,BILATERALITY OF TUMOR_1,BILATERALITY OF TUMOR_2,SIDE OF TUMOR_0,SIDE OF TUMOR_1,SIDE OF TUMOR_2,SIDE OF TUMOR_3,SIDE OF SURGERY_0,SIDE OF SURGERY_1,SIDE OF SURGERY_2,SIDE OF SURGERY_3,FACE_0,FACE_1,FACE_2,TUMOR lOCATION_0,TUMOR lOCATION_1,TUMOR lOCATION_2,TUMOR lOCATION_3,POLAR LOCATION_0,POLAR LOCATION_1,POLAR LOCATION_2,RENAL SINUS_0,RENAL SINUS_1,RENAL SINUS_2,RENAL SINUS_3,RENAL SINUS_4,EXOPHYTIC RATE_0,EXOPHYTIC RATE_1,EXOPHYTIC RATE_2,EXOPHYTIC RATE_3,CLINICAL SIZE GROUP_0,CLINICAL SIZE GROUP_1,CLINICAL SIZE GROUP_2,CLINICAL SIZE GROUP_3,CT_0,CT_1,CT_2,CT_3,CT_4,CT_5,CT_6,CN_0,CN_1,CN_2,CN_3,CM_0,CM_1,CM_2,CM_3,RADIUS (maximal diameter in cm)_0,RADIUS (maximal diameter in cm)_1,RADIUS (maximal diameter in cm)_2,RADIUS (maximal diameter in cm)_3,EXOPHYTIC/ENDOPHYTIC PROPERTIES_0,EXOPHYTIC/ENDOPHYTIC PROPERTIES_1,EXOPHYTIC/ENDOPHYTIC PROPERTIES_2,EXOPHYTIC/ENDOPHYTIC PROPERTIES_3,ANTERIOR OR POSTERIOR_0,ANTERIOR OR POSTERIOR_1,ANTERIOR OR POSTERIOR_2,ANTERIOR OR POSTERIOR_3,PARTIAL NEPHRO INDICATION_0,PARTIAL NEPHRO INDICATION_1,PARTIAL NEPHRO INDICATION_2,PARTIAL NEPHRO INDICATION_3
0,50.0,57.00,168.0,20.20,7.0,13.2,50.0,7800.0,0.63,100.030000,1.0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0
1,69.0,68.60,161.3,26.37,74.0,14.7,44.2,6400.0,0.80,71.119672,1.0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1
2,83.0,83.65,164.0,31.10,80.0,12.5,38.9,5590.0,0.97,54.844682,1.0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0
3,66.0,107.00,180.0,32.78,73.0,17.0,50.8,8000.0,0.93,81.290783,1.0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0
4,61.0,82.90,171.0,28.35,8.0,12.1,37.1,7710.0,1.10,68.053260,1.0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0


In [7]:
# Split data in test and train
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(nn_data, Y, test_size=0.30, random_state=42, stratify = Y)

In [8]:
print("Number of records in trainset {}".format(len(x_train)))
print("Number records in testset {}".format(len(x_test)))

Number of records in trainset 2318
Number records in testset 994


In [9]:
sum(y_train)/len(y_train)

0.0457290767903365

In [10]:
sum(y_test)/len(y_test)

0.04527162977867203

In [11]:
# Standardize the numeric columns
from sklearn.preprocessing import StandardScaler
stdc = StandardScaler()
x_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
x_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
float_col = x_train.select_dtypes('float').columns
x_train[float_col] = pd.DataFrame(stdc.fit_transform(x_train[float_col]),columns=float_col).copy()
x_test[float_col] = pd.DataFrame(stdc.transform(x_test[float_col]),columns=float_col).copy()

H:\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
H:\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [12]:
x_train.head()

,AGE AT SURGERY,WEIGHT,HEIGHT,BMI,CLINICAL SIZE (mm),PRE-OP HB,PRE-OP HT,PRE-OP WBC,PRE-OP CREAT,PRE-OP EGFR,NO OF LESIONS,PROC NAME_0,GENDER_0,GENDER_1,GENDER_2,MARITAL STATUS_0,MARITAL STATUS_1,MARITAL STATUS_2,MARITAL STATUS_3,MARITAL STATUS_4,RACE_0,RACE_1,RACE_2,RACE_3,RACE_4,SYMPTOMS_0,SYMPTOMS_1,SYMPTOMS_2,SYMPTOMS_3,SOLITARY KIDNEY_0,SOLITARY KIDNEY_1,SOLITARY KIDNEY_2,BILATERALITY OF TUMOR_0,BILATERALITY OF TUMOR_1,BILATERALITY OF TUMOR_2,SIDE OF TUMOR_0,SIDE OF TUMOR_1,SIDE OF TUMOR_2,SIDE OF TUMOR_3,SIDE OF SURGERY_0,SIDE OF SURGERY_1,SIDE OF SURGERY_2,SIDE OF SURGERY_3,FACE_0,FACE_1,FACE_2,TUMOR lOCATION_0,TUMOR lOCATION_1,TUMOR lOCATION_2,TUMOR lOCATION_3,POLAR LOCATION_0,POLAR LOCATION_1,POLAR LOCATION_2,RENAL SINUS_0,RENAL SINUS_1,RENAL SINUS_2,RENAL SINUS_3,RENAL SINUS_4,EXOPHYTIC RATE_0,EXOPHYTIC RATE_1,EXOPHYTIC RATE_2,EXOPHYTIC RATE_3,CLINICAL SIZE GROUP_0,CLINICAL SIZE GROUP_1,CLINICAL SIZE GROUP_2,CLINICAL SIZE GROUP_3,CT_0,CT_1,CT_2,CT_3,CT_4,CT_5,CT_6,CN_0,CN_1,CN_2,CN_3,CM_0,CM_1,CM_2,CM_3,RADIUS (maximal diameter in cm)_0,RADIUS (maximal diameter in cm)_1,RADIUS (maximal diameter in cm)_2,RADIUS (maximal diameter in cm)_3,EXOPHYTIC/ENDOPHYTIC PROPERTIES_0,EXOPHYTIC/ENDOPHYTIC PROPERTIES_1,EXOPHYTIC/ENDOPHYTIC PROPERTIES_2,EXOPHYTIC/ENDOPHYTIC PROPERTIES_3,ANTERIOR OR POSTERIOR_0,ANTERIOR OR POSTERIOR_1,ANTERIOR OR POSTERIOR_2,ANTERIOR OR POSTERIOR_3,PARTIAL NEPHRO INDICATION_0,PARTIAL NEPHRO INDICATION_1,PARTIAL NEPHRO INDICATION_2,PARTIAL NEPHRO INDICATION_3
0,0.865790,0.767299,0.489424,0.410944,-0.343254,-0.612757,0.025225,0.063112,0.948651,-1.344635,-0.225829,1,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0
1,1.260646,-0.016891,-0.020296,-0.267679,-0.590435,0.007766,0.025225,-0.003458,-0.208257,-0.865275,-0.028961,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0
2,-0.713633,-0.016891,-0.020296,-0.120950,-0.652230,0.007766,0.025225,-0.003458,0.111738,0.668672,-0.028961,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0
3,0.786819,-0.673305,-1.374067,0.008246,-0.961205,-1.572329,0.025225,-1.944259,-0.035952,-1.057020,-0.225829,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0
4,1.260646,-0.234860,-1.593302,0.680559,0.027516,-0.292900,0.025225,-0.003458,-1.315935,0.022017,-0.225829,1,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0


In [13]:
1/sum(y_train)

0.009433962264150943

In [14]:
1/(len(y_train)-sum(y_train))

0.0004520795660036166

In [15]:
# calculater class weight
class_weight = {0: 1/(len(y_train)-sum(y_train)), 1: 1/sum(y_train)}

In [21]:
# input layer
input_layer = keras.layers.Input(shape=(x_train.shape[1],), name = "input_layer")
x = keras.layers.Dense(256, name = 'Dense_1')(input_layer)
x = keras.layers.Dropout(0.2, name=  'Dropout_1')(x)
x = keras.layers.Dense(256, name = 'Dense_2')(x)
x = keras.layers.Dropout(0.2, name=  'Dropout_2')(x)
main_output = keras.layers.Dense(1, activation='sigmoid',name='main_output')(x)

model = keras.Model(inputs= input_layer, outputs=main_output)

# compiling the model
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(curve = 'ROC',name = 'AUC_ROC'),
                       tf.keras.metrics.AUC(curve = 'PR', name = 'AUC_PR')],
              )

# Keras callback. The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min',restore_best_weights=True)


model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 97)]              0         
_________________________________________________________________
Dense_1 (Dense)              (None, 256)               25088     
_________________________________________________________________
Dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
Dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
Dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
main_output (Dense)          (None, 1)                 257       
Total params: 91,137
Trainable params: 91,137
Non-trainable params: 0
_______________________________________________________

In [22]:
model.fit(x = x_train, y = y_train,
          validation_split = 0.2,
          epochs = 50,
          callbacks = [early_stop]
     )


Train on 1854 samples, validate on 464 samples
Epoch 1/50
1854/1854 [==============================] - 1s 529us/sample - loss: 0.2309 - AUC_ROC: 0.6001 - AUC_PR: 0.0728 - val_loss: 0.1667 - val_AUC_ROC: 0.5651 - val_AUC_PR: 0.0486
Epoch 2/50
1854/1854 [==============================] - 0s 106us/sample - loss: 0.1823 - AUC_ROC: 0.6985 - AUC_PR: 0.0858 - val_loss: 0.1629 - val_AUC_ROC: 0.6308 - val_AUC_PR: 0.0601
Epoch 3/50
1854/1854 [==============================] - 0s 103us/sample - loss: 0.1776 - AUC_ROC: 0.7244 - AUC_PR: 0.0961 - val_loss: 0.1605 - val_AUC_ROC: 0.6599 - val_AUC_PR: 0.0685
Epoch 4/50
1854/1854 [==============================] - 0s 103us/sample - loss: 0.1735 - AUC_ROC: 0.7484 - AUC_PR: 0.1116 - val_loss: 0.1621 - val_AUC_ROC: 0.6761 - val_AUC_PR: 0.0722
Epoch 5/50
1854/1854 [==============================] - 0s 103us/sample - loss: 0.1693 - AUC_ROC: 0.7803 - AUC_PR: 0.1268 - val_loss: 0.1573 - val_AUC_ROC: 0.6948 - val_AUC_PR: 0.0809
Epoch 6/50
1854/1854 [===========

In [23]:
# predict on test set
pd.DataFrame(model.predict(x_test))

,0
0,0.181028
1,0.045467
2,0.052542
3,0.030579
4,0.046233
5,0.034239
6,0.054915
7,0.066150
8,0.066208
9,0.066835


In [24]:
results_nn = pd.DataFrame(model.predict(x_test), columns=['pred_prob'])
results_nn['pred_label'] =  results_nn['pred_prob'].apply(lambda x: 1 if x>=0.5 else 0)
results_nn['true_label'] = np.array(y_test)

In [25]:
# NeuralNetwork Score Raw Data
print("\n Model Balanced Accuracy: \n" + str(metrics.balanced_accuracy_score(results_nn['true_label'], results_nn['pred_label'])))
print("\n Confusion Matrix : \n"+str(metrics.confusion_matrix(results_nn['true_label'], results_nn['pred_label'])))
print("\n Classification Report: \n"+ str(metrics.classification_report(results_nn['true_label'], results_nn['pred_label'])))
print("\n AUC-ROC: \n"+ str(metrics.roc_auc_score(results_nn['true_label'], results_nn['pred_prob'])))


def calc_aucpr_data(result):
    y_ACTUAL = result['true_label']
    scores_prob = result['pred_prob']
    yhat = result['pred_label']
    precision, recall, thresholds = metrics.precision_recall_curve(y_ACTUAL, scores_prob, pos_label=1)
    prc_auc = metrics.auc(recall,precision)
    return prc_auc

print("\n AUC-ROC: \n"+ str(calc_aucpr_data(results_nn)))


 Model Balanced Accuracy: 
0.5

 Confusion Matrix : 
[[949   0]
 [ 45   0]]

 Classification Report: 
              precision    recall  f1-score   support

           0       0.95      1.00      0.98       949
           1       0.00      0.00      0.00        45

    accuracy                           0.95       994
   macro avg       0.48      0.50      0.49       994
weighted avg       0.91      0.95      0.93       994


 AUC-ROC: 
0.7950591265659759

 AUC-ROC: 
0.12691845438090554


H:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [89]:
from sklearn.utils import resample
resample()

In [71]:
def bootstrapped_AUC(result):
    from sklearn.utils import resample
    from tqdm import tqdm

    n_iter = 10000
    roc_auc = list()
    prc_auc = list()


    for i in range(n_iter):
        result_sample = resample(result, n_samples = len(result),random_state=i)
        
        #Calculating AUROC for each sample
        y_ACTUAL= result_sample['true_label']
        scores_prob = result_sample['pred_prob']
        fpr, tpr, thresholds = metrics.roc_curve(y_ACTUAL, scores_prob, pos_label=1)
        roc_auc.append(metrics.auc(fpr, tpr))

        #calculate AUPRC for each sample
        y_ACTUAL = result_sample['true_label']
        scores_prob = result_sample['pred_prob']
        yhat = result_sample['pred_label']
        precision, recall, thresholds = metrics.precision_recall_curve(y_ACTUAL, scores_prob, pos_label=1)
        prc_auc.append(metrics.auc(recall,precision))
    
    return roc_auc, prc_auc

In [72]:
roc_auc_nn, pr_auc_nn = bootstrapped_AUC(results_nn)

In [74]:
dict = {'roc_auc_nn': roc_auc_nn,
        'pr_auc_nn': pr_auc_nn,
       }
pd.DataFrame(dict).describe(percentiles=[0.025,0.975])

,roc_auc_nn,pr_auc_nn
count,10000.000000,10000.000000
mean,0.896966,0.568454
std,0.021756,0.074140
min,0.781824,0.237723
2.5%,0.851586,0.418560
50%,0.897717,0.570469
97.5%,0.936316,0.706917
max,0.969432,0.813393


In [53]:
pd.DataFrame(roc_auc_nn).describe(percentiles=[0.025,0.975])

,0
count,10000.000000
mean,0.885172
std,0.024511
min,0.746168
2.5%,0.833608
50%,0.886348
97.5%,0.929689
max,0.965700


In [22]:
pd.DataFrame(roc_auc_nn).describe(percentiles=[0.025,0.975])

,0
count,10000.000000
mean,0.853840
std,0.028179
min,0.735458
2.5%,0.794303
50%,0.854971
97.5%,0.905273
max,0.941009
